## 1. Install latest SDK

In [1]:
! pip install azureml-sdk[automl,explain,notebooks]==1.0.43

## 2. Load workspace from conf.json

In [3]:
import azureml.core
import pandas as pd
import numpy as np
import logging
import warnings
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None


from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig


In [5]:
ws = Workspace.from_config()



output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

UserErrorException: We could not find config.json in: D:\Dev\AutoML-AIConf-2019-BJ or in its parent directories. Please provide the full path to the config file or ensure that config.json exists in the parent directories.

## 3. Print the exiting compute in the workspace

In [2]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os



compute_target_name='cpucluster'
compute = ws.compute_targets
compute_target=compute[compute_target_name]
print(compute_target.serialize())

NameError: name 'ws' is not defined

## 4. Upload data to blob storage for later on experiments

In [5]:
ds = ws.get_default_datastore()
ds.upload_files(['./data/nyc_energy.csv'], target_path='exp_data', overwrite=True, show_progress=True)

from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='exp_data', 
                   path_on_compute='/tmp/azureml_runs',
                   mode='download', # download files from datastore to compute target
                   overwrite=False)

Uploading ./data/nyc_energy.csv
Uploaded ./data/nyc_energy.csv, 1 files out of an estimated total of 1
